In [71]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import vincenty
import sys

foursquare_data = pd.read_csv("./data/foursquare-nyc-and-tokyo-check-ins/dataset_TSMC2014_NYC.csv")
climate_data = pd.read_csv("./data/1087533.csv")

/Users/frankyliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,10,11,14,15,20,23,25,40,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
pd.options.display.max_columns = 999
climate_data.HOURLYPrecip = climate_data.HOURLYPrecip.fillna(0.00).replace('T', '0.00').str.replace(r"[a-zA-Z]",'').astype(float)

In [73]:
foursquare_data.columns

Index(['userId', 'venueId', 'venueCategoryId', 'venueCategory', 'latitude',
       'longitude', 'timezoneOffset', 'utcTimestamp'],
      dtype='object')

In [79]:
def closest_station(point, points):
    closest_station = ""
    closest_distance = 50000                                                 
    for station in points:
        cur_dist = vincenty(point, station).kilometers
        if cur_dist < closest_distance:
            closest_station = station_lat_long[station_lat_long['lat_long'] == station]['STATION_NAME'].values[0]
    return closest_station

station_lat_long = climate_data[['STATION_NAME', 'LATITUDE', 'LONGITUDE']].drop_duplicates()
climate_data['lat_long'] = [(x, y) for x,y in zip(climate_data['LATITUDE'], climate_data['LONGITUDE'])]
foursquare_data['lat_long'] = [(x, y) for x,y in zip(foursquare_data['latitude'], foursquare_data['longitude'])]                                       
station_lat_long['lat_long'] = [(x, y) for x,y in zip(station_lat_long['LATITUDE'], station_lat_long['LONGITUDE'])]

foursquare_data['closestStation'] = [closest_station(pt, station_lat_long['lat_long']) for pt in foursquare_data['lat_long']]



In [81]:
foursquare_data.sample(1)['closestStation'].item()
climate_data.sample(1)['STATION_NAME'].item()

'LA GUARDIA AIRPORT NY US'

In [82]:
foursquare_data.sample(1)['closestStation'].item()

'JFK INTERNATIONAL AIRPORT NY US'

In [27]:
climate_data.HOURLYPrecip[climate_data.HOURLYPrecip > 0]

229      0.01
230      0.01
232      0.02
234      0.01
349      0.21
350      0.23
351      0.01
352      0.12
354      0.02
368      0.03
369      0.03
370      0.04
371      0.08
373      0.11
374      0.10
375      0.12
377      0.14
378      0.04
380      0.06
381      0.08
382      0.06
383      0.17
384      0.10
385      0.14
386      0.16
387      0.19
389      0.22
391      0.02
392      0.03
394      0.20
         ... 
33383    0.05
33384    0.04
33386    0.01
33387    0.01
33389    0.01
33457    0.03
33458    0.08
33460    0.05
33461    0.02
33462    0.07
33463    0.09
33464    0.12
33465    0.06
33466    0.12
33467    0.11
33469    0.03
33470    0.04
33471    0.04
33473    0.07
33474    0.01
33475    0.01
33479    0.01
33485    0.01
33486    0.01
33487    0.01
33488    0.01
33496    0.01
33497    0.01
33581    0.01
33583    0.01
Name: HOURLYPrecip, Length: 2786, dtype: float64

In [28]:
climate_data.DATE = pd.to_datetime(climate_data.DATE, utc=True)
climate_data.sort_values('DATE', inplace=True)
foursquare_data.utcTimestamp = pd.to_datetime(foursquare_data.utcTimestamp, utc=True)
foursquare_data.sort_values('utcTimestamp', inplace=True)

In [31]:
# climate_data.set_index("DATE", drop=False)
# foursquare_data.set_index("utcTimestamp")
# climate_data.reindex(foursquare_data.index, method='nearest')

cd1 = climate_data.set_index(["STATION_NAME","DATE"], drop=False).reindex(foursquare_data.set_index("utcTimestamp").index, method = "nearest").reset_index()

NotImplementedError: method='nearest' not implemented yet for MultiIndex; see GitHub issue 9365

In [ ]:
print(climate_data)

In [6]:
climate_data.DATE.dt.round('5min')

0       2012-04-12 00:50:00
1       2012-04-12 01:00:00
2       2012-04-12 01:50:00
3       2012-04-12 02:50:00
4       2012-04-12 03:50:00
5       2012-04-12 04:00:00
6       2012-04-12 04:50:00
7       2012-04-12 05:50:00
8       2012-04-12 06:50:00
9       2012-04-12 07:00:00
10      2012-04-12 07:50:00
11      2012-04-12 08:50:00
12      2012-04-12 09:50:00
13      2012-04-12 10:00:00
14      2012-04-12 10:50:00
15      2012-04-12 11:50:00
16      2012-04-12 12:50:00
17      2012-04-12 13:00:00
18      2012-04-12 13:50:00
19      2012-04-12 14:50:00
20      2012-04-12 15:50:00
21      2012-04-12 16:00:00
22      2012-04-12 16:50:00
23      2012-04-12 17:50:00
24      2012-04-12 18:50:00
25      2012-04-12 19:00:00
26      2012-04-12 19:50:00
27      2012-04-12 20:50:00
28      2012-04-12 21:50:00
29      2012-04-12 22:00:00
                ...        
33659   2013-02-16 04:50:00
33660   2013-02-16 05:50:00
33661   2013-02-16 05:50:00
33662   2013-02-16 06:50:00
33663   2013-02-16 0

In [9]:
 foursquare_data.utcTimestamp = pd.to_datetime(foursquare_data.utcTimestamp).dt.round('5min')

In [10]:
lga = climate_data.loc[climate_data.STATION_NAME == "LA GUARDIA AIRPORT NY US"]
foursquare_join_lga = pd.concat([foursquare_data.set_index('utcTimestamp'), lga.set_index('DATE')], axis=1, join='inner')
foursquare_join_climate

ValueError: Shape of passed values is (96, 56111), indices imply (96, 6849)